# Section: BLAS/LAPACK - Linear Equations


Adapted from: [https://github.com/gjbex/Fortran-MOOC/tree/master/source_code/blas_lapack/linear_equations](https://github.com/gjbex/Fortran-MOOC/tree/master/source_code/blas_lapack/linear_equations)

## This program demonstrates solving linear algebra equations in Fortran.

### Linear Equation Problem

In this notebook we will use Fortran to solve a system of linear equations of the form:

$$
\Large A \mathbf{x} = \mathbf{b}
$$
where: <br>


$$
\Large A =
\left[
\begin{array}{cccc}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \vdots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{array}
\right]
$$

$$
\Large \mathbf{x} =
\left[
\begin{array}{c}
x_{1}  \\
x_{2}  \\
\vdots \\
x_{n}
\end{array}
\right]
$$

$$
\Large \mathbf{b} = 
\left[
\begin{array}{c}
b_{1}  \\
b_{2}  \\
\vdots \\
b_{n}
\end{array}
\right]
$$

The approach will be to have a main program that reads in as parameters the number of equations, the matrix $\mathbf{A}$ and the vector $\mathbf{x}$ as text files.  The main program is called *solve_equations.f90*.  It makes use of the LAPACK linear equation solver routine called **DGESV**.  Information on this routine can be found at [LAPACK - DGESV](https://netlib.org/lapack/explore-html/d7/d3b/group__double_g_esolve_ga5ee879032a8365897c3ba91e3dc8d512.html#ga5ee879032a8365897c3ba91e3dc8d512).

The main program makes use of a module called *linalg_mod.f90*, which contains subroutines to read and write matrices and arrays to and from text files.

Finally, there is a helper program called *generate_array.f90* which has a subroutine that gets arguments from the command line and generates either of vector or matrix of specified size.  Every element in the array or matrix is a random number.

The individual program files are listed below:

### In file linalg_mod.f90

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/src/linalg_mod.f90
---
language: fortran
---
```

### In file generate_array.f90

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/app/generate_array.f90
---
language: fortran
---
```

### In solve_equations.f90

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/app/solve_equations.f90
---
language: fortran
---
```

The above programs are compiled and run using Fortran Package Manager (fpm):

## Build the Program using FPM (Fortran Package Manager)

In [2]:
import os
root_dir = ""
root_dir = os.getcwd()

Since the code makes use of the LAPACK library, the following FPM configuration file (fpm.toml) was used:

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/fpm.toml
---
language: toml
---
```

In [3]:
code_dir = root_dir + "/" + "Fortran_Code/Section_BLAS_LAPACK_Linear_Equations"

In [4]:
os.chdir(code_dir)

The files *solve_equations.f90* and *generate_array.f90* were placed into the "app" folder, while the file *linalg_mod.f90* was placed into the "src" folder.

In [5]:
build_status = os.system("fpm build 2>/dev/null")

## Run the Program using FPM (Fortran Package Manager)

### Solve a Test Linear System of Two Equations

As our first run, we wish to solve the following set of linear equations:

$$
\begin{align*}
2x+8y & = 20 \\
x+2y  & = 4
\end{align*}
$$

The variables in the equations are converted into components of the $\mathbf{x}$ vector as shown below:

$$
\begin{align*}
2x_1+8x_2 & = 20 \\
x_1+2x_2  & = 4
\end{align*}
$$

These equations are converted into matrix form as shown below:

$$
\begin{equation*}
\left[
\begin{array}{cc}
2 & 8 \\
1 & 2 \\
\end{array}
\right]
\left[
\begin{array}{c}
x_1 \\
x_2 \\
\end{array}
\right]
=
\left[
\begin{array}{c}
20 \\
4 \\
\end{array}
\right]
\end{equation*}
$$

Therefore we have the following:

$$
\mathbf{A} = 
\left[
\begin{array}{cc}
2 & 8 \\
1 & 2 
\end{array}
\right]
$$

$$
\mathbf{x} = 
\left[
\begin{array}{c}
x_1 \\
x_2  
\end{array}
\right]
$$


$$
\mathbf{b} = 
\left[
\begin{array}{c}
20 \\
4  
\end{array}
\right]
$$


The matrix $\mathbf{A}$ and the vector $\mathbf{b}$ are written into text files as shown below:

In [6]:
%%writefile A_test1.txt
2 8
1 2

Overwriting A_test1.txt


In [7]:
%%writefile b_test1.txt
20
4

Overwriting b_test1.txt


The *solve_equations* program can now be run with the number of equations command line argument set to 2, and the files *A_test1.txt* and *b_test1.txt*

In [8]:
exec_status = \
    os.system("fpm run solve_equations 2>/dev/null -- 2 A_test1.txt b_test1.txt")

     -0.200000000000000E+01
      0.300000000000000E+01


The results are printed in scientfic notation and in the order of $x_1$, $x_2$.

We now wish to use Python's Numpy library to test these results:

In [9]:
import numpy as np

A = np.genfromtxt("A_test1.txt")
b = np.genfromtxt("b_test1.txt")
x = np.linalg.solve(A, b)
print("x1 = {0:2.1f}".format(x[0]))
print("x2 = {0:2.1f}".format(x[1]))

x1 = -2.0
x2 = 3.0


We can see that the Fortran code and Numpy produce the same results.

### Solve a Test Linear System of Equations of Arbitrary Size

The Fortran code can be used to solve an arbitrarily large system of equations.  To test this functionality, we make use of the *generate_array.f90* program to generate arrays or matrices of arbitrary size filled with random numbers.

As a start, we will use the *generate_array.f90* to generate a matrix file A_test2.txt that contains a 10x10 matrix.

In [10]:
exec_status = os.system("fpm run generate_array 2>/dev/null -- 10 10 > A_test2.txt") 

The $\mathbf{A}$ matrix is shown below:

In [11]:
import pandas as pd
A = pd.read_table("A_test2.txt", 
    header=None, 
    sep='\s+')
A

,0,1,2,3,4,5,6,7,8,9
0,0.277215,0.806160,0.559751,0.421605,0.661636,0.175702,0.280412,0.788789,0.730208,0.940157
1,0.988067,0.100495,0.508618,0.191515,0.666810,0.681284,0.237398,0.476677,0.745980,0.009742
2,0.307970,0.242769,0.058610,0.331152,0.245847,0.106706,0.387305,0.329858,0.726194,0.195144
3,0.831244,0.768113,0.198732,0.448814,0.270868,0.318633,0.545194,0.271570,0.917646,0.306933
4,0.118196,0.879517,0.801568,0.179744,0.553822,0.342631,0.871735,0.604017,0.637519,0.717799
5,0.470730,0.573523,0.070252,0.545145,0.967696,0.357509,0.601256,0.152319,0.214831,0.528039
6,0.536436,0.957059,0.208592,0.922741,0.290713,0.955667,0.696410,0.275372,0.197347,0.977716
7,0.370497,0.230169,0.713194,0.094943,0.667556,0.845524,0.647557,0.015771,0.880148,0.072044
8,0.298487,0.808056,0.539075,0.689543,0.742135,0.987072,0.127719,0.043992,0.194097,0.894006
9,0.244722,0.815570,0.104419,0.152344,0.144090,0.999296,0.629343,0.632703,0.804412,0.502865


And now we generate the $\mathbf{b}$ vector:

In [12]:
exec_status = os.system("fpm run generate_array 2>/dev/null -- 10 > b_test2.txt") 

The $\mathbf{b}$ vector is shown below:

In [13]:
b = pd.read_table("b_test2.txt", 
    header=None, 
    sep='\s+')
b

,0
0,0.914628
1,0.261707
2,0.588643
3,0.334762
4,0.600650
5,0.724658
6,0.723074
7,0.347790
8,0.495246
9,0.771957


We now use the *solve_equation* Fortran code to solve this linear system of equations:

In [14]:
exec_status = \
    os.system("fpm run solve_equations 2>/dev/null -- 10 A_test2.txt b_test2.txt")

     -0.202071285945788E+00
     -0.633784665553714E+00
     -0.631080704368926E+00
     -0.208873435610023E-01
      0.335360198881879E+00
      0.993230206615972E-01
      0.315450977232440E+00
      0.229422519458424E+00
      0.488438759811549E+00
      0.104047447511920E+01


And the results are compared to the output of Numpy:

In [17]:
A = np.genfromtxt("A_test2.txt")
b = np.genfromtxt("b_test2.txt")
x = np.linalg.solve(A, b)

for i in range(len(x)):
    print ("x{0:d} = {1:2.6f}".format(i+1, x[i]))

x1 = -0.202071
x2 = -0.633785
x3 = -0.631081
x4 = -0.020887
x5 = 0.335360
x6 = 0.099323
x7 = 0.315451
x8 = 0.229423
x9 = 0.488439
x10 = 1.040474


Again, we see that the results are the same.